<a href="https://colab.research.google.com/github/gizdatalab/CPU/blob/main/classifier/TAPP/CPU_TAPP_SentenceT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! pip install datasets transformers sentencepiece huggingface_hub
! apt install git-lfs
! pip install sentence-transformers
! pip install optuna
! pip install evaluate
! pip install accelerate -U
! pip install codecarbon

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

[codecarbon INFO @ 15:43:11] Energy consumed for RAM : 0.000137 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 15:43:11] Energy consumed for all GPUs : 0.000123 kWh. Total GPU Power : 9.28099058829887 W
[codecarbon INFO @ 15:43:11] Energy consumed for all CPUs : 0.000565 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:43:11] 0.000825 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:43:56] Energy consumed for RAM : 0.000265 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 15:43:56] Energy consumed for all GPUs : 0.000414 kWh. Total GPU Power : 25.441500917784857 W
[codecarbon INFO @ 15:43:56] Energy consumed for all CPUs : 0.001096 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:43:56] 0.001776 kWh of electricity used since the beginning.


In [ ]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()
tracker.start()

[codecarbon INFO @ 15:42:21] [setup] RAM Tracking...
[codecarbon INFO @ 15:42:21] [setup] GPU Tracking...
[codecarbon INFO @ 15:42:21] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 15:42:21] [setup] CPU Tracking...
[codecarbon WARNING @ 15:42:21] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 15:42:22] We saw that you have a AMD EPYC 7V12 64-Core Processor but we don't know it. Please contact us.
[codecarbon INFO @ 15:42:22] CPU Model on constant consumption mode: AMD EPYC 7V12 64-Core Processor
[codecarbon INFO @ 15:42:23] >>> Tracker's metadata:
[codecarbon INFO @ 15:42:23]   Platform system: Linux-5.15.0-1040-azure-x86_64-with-glibc2.10
[codecarbon INFO @ 15:42:23]   Python version: 3.8.5
[codecarbon INFO @ 15:42:23]   CodeCarbon version: 2.3.2
[codecarbon INFO @ 15:42:23]   Available RAM : 27.406 GB
[codecarbon INFO @ 15:42:23]   CPU count: 4
[codecarbon INFO @ 15:42:23]   CPU model: AMD EPYC 7V12 64-Core Processor
[codecarbon INFO @ 15:42:

In [ ]:
from datasets import load_dataset
import datasets
import numpy as np
import pandas as pd
import evaluate
import torch
import os
import sklearn.metrics as skm
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
                          PreTrainedModel, BertModel, BertForSequenceClassification,
                          RobertaForSequenceClassification,
                          TrainingArguments, Trainer, TrainerCallback)
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

2024-01-18 15:42:35.666539: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-18 15:42:38.954607: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-18 15:42:38.954632: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[codecarbon INFO @ 15:42:41] Energy consumed for RAM : 0.000051 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 15:42:41] Energy consumed for all GPUs : 0.000046 kWh. Total GPU Power : 9.318893677871012 W
[codecarbon INFO @ 15:42:41] Ene

# Manual Inputs


In [ ]:
###### Define Path to data ############################
repo_id = "GIZ/policy_classification"
filename="policy_classification.json"


### Define labels ##################
label_names= ['TargetLabel','ActionLabel','PlansLabel','PolicyLabel']


#######  Define Model params ######################

# define the model checkpoint
model_checkpoint = "sentence-transformers/all-mpnet-base-v2"

# define the problem type
problem_type="multi_label_classification"


######## Training Arguments ##################
output_dir = "TAPP-multilabel-mpnet"

# logging and evaluation
evaluation_strategy = "epoch"
logging_strategy="epoch"
logging_steps = 1
eval_steps = 1

# Important params
learning_rate=5.86e-5   #4.04e-5
batch_size = 8
num_train_epochs= 10
weight_decay=0.05
warmup_steps = 100
gradient_accumulation_steps = 2

[codecarbon INFO @ 15:42:56] Energy consumed for RAM : 0.000094 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 15:42:56] Energy consumed for all GPUs : 0.000085 kWh. Total GPU Power : 9.205770481200863 W
[codecarbon INFO @ 15:42:56] Energy consumed for all CPUs : 0.000388 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:42:56] 0.000567 kWh of electricity used since the beginning.


# Automated

## Dataset

In [ ]:
# Load Dataset from Hugging Face
# Load Dataset from Hugging Face
try:
    dataset = load_dataset(repo_id, data_files=filename,
                             use_auth_token=True )
    data = pd.DataFrame(dataset['train'])
except:
    from huggingface_hub import hf_hub_download
    hf_hub_download(repo_id, filename=filename,
                    repo_type="dataset",use_auth_token=True,
    local_dir= 'data/')
    data = pd.read_json('data/policy_classification.json')

# the dataset already contains train_test split flag we use
# it to create train-test split.

train_df = data[data.split== 'Train'].reset_index(drop=True)
test_df = data[data.split=='Test'].reset_index(drop=True)

# Sort the labels and create plaecholders for id2label and label2id
label_names.sort()

num_labels = len(label_names)
id2label = {idx:label for idx, label in enumerate(label_names)}
label2id = {label:idx for idx, label in enumerate(label_names)}
print(id2label)

Using custom data configuration GIZ--policy_classification-cdc4b717b1bd720b


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Failed to read file '/home/azureuser/.cache/huggingface/datasets/downloads/dee24a3afc63cd87c4a0a9517165e43822030f22df905fa089a2628e8d786ca4' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Column() changed from object to array in row 0


{0: 'ActionLabel', 1: 'PlansLabel', 2: 'PolicyLabel', 3: 'TargetLabel'}


In [ ]:
def prepare_multilabel_df(df, cols):
    """
    Takes the dataframe and list of columns which need to combined into
    One-hot-encoded labels.

    """
    df.dropna(subset = cols, inplace=True)
    df = df.reset_index(drop=True)
    df['labels'] = df.apply(lambda x: [int(x[label]) for label in cols],axis=1)
    # we have multilingual dataset ('en', 'fr','es'), in case of non-english the
    # translated Context is udner the column 'TranslatedContext'. We impute these
    # back into 'context' to work with only English dataset.
    df['Context'] = df.apply(lambda x: x['Context'] if x['Language'] == 'en'
                                    else x['TranslatedContext'][0],axis=1)
    df.rename(columns = {'Context':'text'}, inplace=True)

    df = df[['text','labels']+cols]
    return df

# creating multilabel dataset
train_df = prepare_multilabel_df(train_df, label_names)
test_df = prepare_multilabel_df(test_df, label_names)

print('\n','Classes Representation in Training Dataset:',len(train_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(train_df['labels'])]))

print('\n','Classes Representation in Test Dataset:',len(test_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(test_df['labels'])]))


 Classes Representation in Training Dataset: 10031
0 . ActionLabel : 5416
1 . PlansLabel : 2140
2 . PolicyLabel : 1396
3 . TargetLabel : 2911

 Classes Representation in Test Dataset: 1121
0 . ActionLabel : 600
1 . PlansLabel : 237
2 . PolicyLabel : 152
3 . TargetLabel : 322


In [ ]:
for i,label in enumerate(label_names):
    # df_train[sector] = df_train.apply(lambda x: x['sector_label'][i], axis =1)
    print(i,".",label, ":", sum(train_df[label]))

# these are weights which can be used in building weighted loss if class is imbalanced
positive_weights = {}
negative_weights = {}
for label in label_names:
    positive_weights[label] = train_df.shape[0]/(2*np.count_nonzero(train_df[label]==True))
    negative_weights[label] = train_df.shape[0]/(2*np.count_nonzero(train_df[label]==False))
print("Positive Weights",'\n',positive_weights)
print("Negative Weights",'\n',negative_weights)

# as we dont want to miss anything (high recall) we use positive weights calculated above.
pos_weights = list(positive_weights.values())

# If using GPU we need to place all required data on else there will be error
posweights = torch.FloatTensor(pos_weights).to(device)

0 . ActionLabel : 5416.0
1 . PlansLabel : 2140.0
2 . PolicyLabel : 1396.0
3 . TargetLabel : 2911.0
Positive Weights 
 {'ActionLabel': 0.9260524372230429, 'PlansLabel': 2.3436915887850467, 'PolicyLabel': 3.5927650429799427, 'TargetLabel': 1.7229474407420131}
Negative Weights 
 {'ActionLabel': 1.0867822318526543, 'PlansLabel': 0.6355975161576479, 'PolicyLabel': 0.580833815865663, 'TargetLabel': 0.7044241573033708}


[codecarbon INFO @ 15:43:26] Energy consumed for RAM : 0.000179 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 15:43:26] Energy consumed for all GPUs : 0.000203 kWh. Total GPU Power : 19.134646946579277 W
[codecarbon INFO @ 15:43:26] Energy consumed for all CPUs : 0.000742 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:43:26] 0.001125 kWh of electricity used since the beginning.


`pos_weight > 1 will increase the recall while pos_weight < 1 will increase the precision.`

## Model

In [ ]:
# problem_type, is not needed in tokenizer but keeping it for conformity
# https://huggingface.co/docs/transformers/main_classes/configuration?highlight=multi_label_classification#transformers.PretrainedConfig
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,problem_type=problem_type)

# Set the model config and assign it and device to model
config = AutoConfig.from_pretrained(model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id, problem_type=problem_type)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,config=config).to(device)

Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# we need datasets format to work with
train_ds = datasets.Dataset.from_pandas(train_df[['text','labels']])
test_ds =  datasets.Dataset.from_pandas(test_df[['text','labels']])
train_ds = train_ds.shuffle(seed=7)
test_ds = test_ds.shuffle(seed=7)


# Need to tokenize the data using the tokenizer of the model
def tokenize_and_encode(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,
                        max_length=384)

cols = train_ds.column_names
cols.remove("labels")
print('Training data:',train_ds.num_rows)
print('Validation data:',test_ds.num_rows)

train_tokenized = train_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)
val_tokenized = test_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)

# need this to avoid error due to type mismatch
# https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3
train_tokenized.set_format("torch")
train_tokenized = (train_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

val_tokenized.set_format("torch")
val_tokenized = (val_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

Parameter 'function'=<function tokenize_and_encode at 0x7fa600575820> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Training data: 10031
Validation data: 1121


  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/10031 [00:00<?, ?ex/s]

[codecarbon INFO @ 15:43:41] Energy consumed for RAM : 0.000222 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 15:43:41] Energy consumed for all GPUs : 0.000308 kWh. Total GPU Power : 25.318178707063595 W
[codecarbon INFO @ 15:43:41] Energy consumed for all CPUs : 0.000919 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:43:41] 0.001450 kWh of electricity used since the beginning.


  0%|          | 0/1121 [00:00<?, ?ex/s]

## Metrics

In [ ]:
multilabel_f1_metric = evaluate.load("f1", "multilabel")
multilabel_recall_metric = evaluate.load("recall","multilabel")
multilabel_precision_metric = evaluate.load("precision","multilabel")

def get_scores(y_pred, y_test, thresh=0.5, sigmoid=True):
    """
    Takes the Predictions and Truth Values, and return multilabel metrics
    aggregated on multiple strategies['micro', 'sample', 'weighted']

    """
    # we get raw function value, therefore we apply sigmoid to get to the
    # probabilities
    if problem_type == "multi_label_classification":
        y_pred = torch.from_numpy(y_pred)
        y_test = torch.from_numpy(y_test)
        if sigmoid:
          y_pred = y_pred.sigmoid()
          y_pred = (y_pred > thresh)

        return {
            "precision-micro": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="micro")["precision"],
            "precision-samples": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="samples")["precision"],
            "precision-weighted": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="weighted")["precision"],
            "recall-micro": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="micro")["recall"],
            "recall-samples": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="samples")["recall"],
            "recall-weighted": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="weighted")["recall"],
            "f1-micro": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="micro")["f1"],
            "f1-samples": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="samples")["f1"],
            "f1-weighted": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="weighted")["f1"],
            }
    else:
        y_pred = torch.from_numpy(y_pred).argmax(dim=1)
        y_test = torch.from_numpy(y_test)

        report = skm.classification_report(y_test, y_pred, output_dict=True)
        df_report = pd.DataFrame(report).transpose()
        return {"Precision_macro": df_report.loc['macro avg']['precision'],
                "Precision_weighted": df_report.loc['weighted avg']['precision'],
                "Recall_macro": df_report.loc['macro avg']['recall'],
                "Recall_weighted": df_report.loc['weighted avg']['recall'],
                "F1-Score": df_report.loc['macro avg']['f1-score'],
                "Accuracy": skm.accuracy_score(y_test, y_pred)}


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return get_scores(predictions, labels)

## Training arguments

In [ ]:
args = TrainingArguments(
    output_dir,
    logging_strategy=logging_strategy,
    evaluation_strategy = evaluation_strategy,
    save_strategy = "epoch",
    logging_steps = logging_steps,
    eval_steps= eval_steps,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    lr_scheduler_type = "linear",
    gradient_accumulation_steps = gradient_accumulation_steps,
    warmup_steps = warmup_steps,
)

## Custom Trainer

In [ ]:
# for class weights we need to use Custom Multi-Label Trainer
# In multi-label problem we will be using Binary Cross Entropy loss with
# sigmoid layer on top rather than softmax.
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=posweights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels),
                        labels.float().view(-1, self.model.config.num_labels))
        return (loss, outputs) if return_outputs else loss


# Training

In [ ]:
multi_trainer =  MultilabelTrainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)


[codecarbon INFO @ 15:43:48] [setup] RAM Tracking...
[codecarbon INFO @ 15:43:48] [setup] GPU Tracking...
[codecarbon INFO @ 15:43:48] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 15:43:48] [setup] CPU Tracking...
[codecarbon WARNING @ 15:43:48] No CPU tracking mode found. Falling back on CPU constant mode.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[codecarbon WARNING @ 15:43:49] We saw that you have a AMD EPYC 7V12 64-Core Processor but we don't know it. Please contact us.
[codecarbon INFO @ 15:43:49] CPU Model on constant consumption mode: AMD EPYC 7V12 64-Core Processor
[codecarbon INFO @ 15:43:49] >>> Tracker's metadata:
[codecarbon INFO @ 15:43:49]   Platform system: Linux-5.15.0-1040-azure-x86_64-w

In [ ]:
multi_trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
You're using a MPNetTokenize

Epoch,Training Loss,Validation Loss,Precision-micro,Precision-samples,Precision-weighted,Recall-micro,Recall-samples,Recall-weighted,F1-micro,F1-samples,F1-weighted
1,0.614100,0.540921,0.663230,0.706809,0.674004,0.736079,0.730746,0.736079,0.697758,0.695710,0.700565
2,0.466100,0.495837,0.720812,0.722272,0.738981,0.758200,0.738329,0.758200,0.739033,0.710599,0.744857
3,0.353700,0.475146,0.720721,0.729111,0.743453,0.793288,0.767916,0.793288,0.755265,0.729311,0.760340
4,0.261800,0.509546,0.764368,0.761374,0.763067,0.811594,0.785905,0.811594,0.787273,0.757402,0.786438
5,0.199000,0.549419,0.784810,0.765016,0.782308,0.803966,0.777059,0.803966,0.794273,0.755907,0.792700
6,0.138100,0.569170,0.773392,0.760184,0.786142,0.807018,0.775647,0.807018,0.789847,0.753133,0.794258
7,0.091900,0.572991,0.785000,0.773788,0.787222,0.838291,0.801293,0.838291,0.810771,0.772648,0.811782
8,0.057800,0.616804,0.785509,0.765239,0.792221,0.835240,0.793562,0.835240,0.809612,0.765549,0.811931
9,0.033600,0.667385,0.780181,0.772822,0.783248,0.852784,0.811106,0.852784,0.814869,0.775434,0.816082
10,0.020900,0.683927,0.800580,0.782263,0.801992,0.842105,0.804044,0.842105,0.820818,0.778455,0.821414


[codecarbon INFO @ 15:44:11] Energy consumed for RAM : 0.000308 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 15:44:11] Energy consumed for all GPUs : 0.000559 kWh. Total GPU Power : 34.64140622178813 W
[codecarbon INFO @ 15:44:11] Energy consumed for all CPUs : 0.001274 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:44:11] 0.002140 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:44:20] Energy consumed for RAM : 0.000043 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 15:44:20] Energy consumed for all GPUs : 0.000255 kWh. Total GPU Power : 61.093521450198004 W
[codecarbon INFO @ 15:44:20] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:44:20] 0.000475 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:44:26] Energy consumed for RAM : 0.000351 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 15:44:26] Energy consumed for all GPUs : 0.000845 kWh. Total GPU Power : 68.7435847208257 W


TrainOutput(global_step=6270, training_loss=0.22370102698342842, metrics={'train_runtime': 8784.3655, 'train_samples_per_second': 11.419, 'train_steps_per_second': 0.714, 'total_flos': 1.979485792607232e+16, 'train_loss': 0.22370102698342842, 'epoch': 10.0})

In [ ]:
tracker.stop()

[codecarbon INFO @ 18:20:36] Energy consumed for RAM : 0.027090 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 18:20:36] Energy consumed for all GPUs : 0.161137 kWh. Total GPU Power : 26.36140900402067 W
[codecarbon INFO @ 18:20:36] Energy consumed for all CPUs : 0.112066 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:20:36] 0.300293 kWh of electricity used since the beginning.


0.10975005264866937

In [ ]:
multi_trainer.push_to_hub()

events.out.tfevents.1705592645.prashant-t4-core4.4285.0:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.14k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

'https://huggingface.co/ppsingh/TAPP-multilabel-mpnet/tree/main/'

In [ ]:
log_results = multi_trainer.state.log_history
print(log_results)

import json
with open("classifier_info/TAPP_mpnet_logs.json", "w") as fp:
    json.dump(log_results , fp)

[{'loss': 0.6141, 'learning_rate': 5.35947811993517e-05, 'epoch': 1.0, 'step': 627}, {'eval_loss': 0.5409209132194519, 'eval_precision-micro': 0.6632302405498282, 'eval_precision-samples': 0.7068093963722866, 'eval_precision-weighted': 0.6740037952411381, 'eval_recall-micro': 0.7360793287566743, 'eval_recall-samples': 0.7307463574189711, 'eval_recall-weighted': 0.7360793287566743, 'eval_f1-micro': 0.697758496023138, 'eval_f1-samples': 0.6957096130155899, 'eval_f1-weighted': 0.7005652103037378, 'eval_runtime': 25.8953, 'eval_samples_per_second': 43.29, 'eval_steps_per_second': 5.445, 'epoch': 1.0, 'step': 627}, {'loss': 0.4661, 'learning_rate': 4.763980551053484e-05, 'epoch': 2.0, 'step': 1254}, {'eval_loss': 0.4958369731903076, 'eval_precision-micro': 0.7208121827411168, 'eval_precision-samples': 0.7222717811477847, 'eval_precision-weighted': 0.7389811255891985, 'eval_recall-micro': 0.7581998474446987, 'eval_recall-samples': 0.7383288730300328, 'eval_recall-weighted': 0.758199847444698

# Predict and Evaluate

## Trainer

In [ ]:
predictions= multi_trainer.predict(val_tokenized)
pred,labels,_ = predictions
y_pred = torch.from_numpy(pred)
y_true = torch.from_numpy(labels)
y_prob = y_pred.sigmoid()
thresh = 0.50
y_pred = (y_prob>=thresh).bool()

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(


In [ ]:
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

ActionLabel
[[431  90]
 [ 77 523]] 

PlansLabel
[[795  89]
 [ 70 167]] 

PolicyLabel
[[928  41]
 [ 28 124]] 

TargetLabel
[[744  55]
 [ 32 290]] 



In [ ]:
report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,ActionLabel,0.853181,0.871667,0.862325,600.0
1,PlansLabel,0.652344,0.704641,0.677485,237.0
2,PolicyLabel,0.751515,0.815789,0.782334,152.0
3,TargetLabel,0.840580,0.900621,0.869565,322.0
4,micro avg,0.800580,0.842105,0.820818,1311.0
5,macro avg,0.774405,0.823180,0.797927,1311.0
6,weighted avg,0.801992,0.842105,0.821414,1311.0
7,samples avg,0.782263,0.804044,0.778455,1311.0
